In [11]:
import numpy as np
import time

# 生成测试数据
n_arrays = 1000
array_length = 1000

# 创建一个包含1000个长度为1000的numpy数组的数组，dtype为float32
data = np.array([np.random.rand(array_length).astype(np.float64) for _ in range(n_arrays)], dtype=object)

# 计时函数
def benchmark_conversion(method, data):
    start_time = time.time()
    result = method(data)
    end_time = time.time()
    return result, end_time - start_time

# 定义各个方法
methods = {
    "concatenate": lambda x: np.concatenate(x).reshape(n_arrays, array_length),
    "stack": lambda x: np.stack(x),
    "vstack": lambda x: np.vstack(x)
}

# 进行benchmark
benchmark_results = {name: benchmark_conversion(method, data) for name, method in methods.items()}

# 返回耗时结果
benchmark_times = {name: time for name, (result, time) in benchmark_results.items()}
benchmark_times


{'concatenate': 0.0022644996643066406,
 'stack': 0.006102323532104492,
 'vstack': 0.004021406173706055}

In [2]:
import pandas as pd
# %cd ..

In [4]:
sunpos_filename = 'data/TUDelft/output_20240902_125140/intermediate/sun_pos.csv'
ground_recording_filename = 'data/TUDelft/measured_data.csv'

In [20]:
sunpos = pd.read_csv(sunpos_filename)
sunpos.columns.values[0]='timestamp'
ground_recording = pd.read_csv(ground_recording_filename)
sunpos['timestamp'] = pd.to_datetime(sunpos['timestamp']).dt.tz_localize(None)
ground_recording['local_time'] = pd.to_datetime(ground_recording['local_time'])
filtered_ground_recording = ground_recording[ground_recording['local_time'].isin(sunpos['timestamp'])]
filtered_sunpos = sunpos[sunpos['timestamp'].isin(ground_recording['local_time'])]

In [ ]:


# 转换 df2 的 local_time 为 datetime 类型


# 过滤 df2，只保留与 df1['timestamp'] 匹配的行
filtered_df2 = df2[df2['local_time'].isin(df1['timestamp'])]

In [ ]:
sunpos = pd.read_csv(sunpos_filename)
sunpos.columns.values[0]='timestamp'

irradiance_data = epw[0]
irradiance_data.index = pd.to_datetime(irradiance_data.index)

sunpos['timestamp'] = pd.to_datetime(sunpos['timestamp'], utc=True)
all_ghi = []
all_dni = []
all_dhi = []
# all_solar_zenith = []
# all_solar_azimuth = []
# all_dni_extra = []

for index, row in sunpos.iterrows():
        row_time = row['timestamp'].tz_convert(irradiance_data.index.tz)
        month, day, hour = row_time.month, row_time.day, row_time.hour
        if month == 2 and day == 29:
            day = 28
        match = irradiance_data[(irradiance_data.index.month == month) & 
                    (irradiance_data.index.day == day) & 
                    (irradiance_data.index.hour == hour)]
        # solar_zenith = row['apparent_zenith']
        # solar_azimuth = row['azimuth']
        # dni_extra = pvlib.irradiance.get_extra_radiation(row['timestamp'])
        if (not match.empty):
            ghi, dni, dhi, dni_extra = match.iloc[0][['ghi', 'dni', 'dhi', 'etrn']]
            if dni_extra == 0:
                dni_extra = pvlib.irradiance.get_extra_radiation(row['timestamp'])
            all_ghi.append(ghi)
            all_dni.append(dni)
            all_dhi.append(dhi)
            # all_solar_zenith.append(solar_zenith)
            # all_solar_azimuth.append(solar_azimuth)
            # all_dni_extra.append(dni_extra)
        else:
            print("error finding match")

tud_data = np.column_stack((all_ghi, all_dhi, all_dni))